# Setup


In [ ]:
%uv pip install mp-api

To carry out this tutorial, you will need your MP API key. Log onto the Materials Projet, go to your [account dashboard](https://next-gen.materialsproject.org/dashboard), and you will find the API key.


In [ ]:
from mp_api.client import MPRester

api_key = ""  # replace with your own API key


The MP API [documentation](https://docs.materialsproject.org/downloading-data/how-do-i-download-the-materials-project-database) is a useful reference if needed.


# Retrieving data using MPRester

- The `MPRester` contains many convenience functions for getting many common types of data such as crystal structures
 - examples are listed [here](https://docs.materialsproject.org/downloading-data/using-the-api/examples)
- Every material on the Materials Project has a unique `material_id`, which can be readily viewed on the Materials Project website
- To start, we will use some known `material_id` to get data for that material


In [ ]:
# Retrieve the crystal structure for a specific material by its Materials Project ID

material_id = "mp-169"  # Example material ID for C

# Fetch structure for the material
with MPRester(api_key=api_key) as mpr:
    structure = mpr.get_structure_by_material_id(material_id)

# Print fetched data
print(structure)

In [ ]:
# Write out structure to CIF to view in VESTA
structure.to(filename=f"{material_id}.cif")


In [ ]:
# Get some other properties by material ID
with MPRester(api_key=api_key) as mpr:
    rho = mpr.get_charge_density_from_material_id(material_id)

rho.write_file(f"CHGCAR_{material_id}")  # CHGCAR tells VESTA it is an electron density

# Using Sub-Resters for Additional Functionality

- The built-in convenience functions of `MPRester` are just a small fraction of the functionality available through the Materials Project REST API
- To access other functions and data as well as to search for materials, you need to use one of the many end points
- Each end point helps retrieve a particular type of data from the Materials Project using one of the REST API endpoints. A full list can be found [here](https://docs.materialsproject.org/downloading-data/using-the-api/getting-started)


- One of the most common end point you may want to use is the `materials/summary` end point, which produces a `SummaryDoc` that contains summary information about materials and their properties
- To see all the available search keyword arguments, "inspect" the `.search` method in VS Code


In [ ]:
# Query the summary doc for two materials
with MPRester(api_key=api_key) as mpr:
    docs = mpr.materials.summary.search(material_ids=["mp-169", "mp-123"])


In [ ]:
# Print out the first `SummaryDoc`

print(docs[0])

In [ ]:
# Query the SummaryDoc
# For a list of all the attributes, do `dir(docs[0])` or (better yet) view the SummaryDoc type hint
print(docs[0].formula_pretty)
print(docs[0].density)
print(docs[0].band_gap)


In [ ]:
# Now let's do a more complex query without relying on a specific MPID
with MPRester(api_key=api_key) as mpr:
    docs = mpr.materials.summary.search(
        elements=["Fe", "O"], exclude_elements=["H"], formation_energy=(-1, 0)
    )

In [ ]:
print(
    f"Found {len(docs)} iron oxides without hydrogen with formation energy between -1 and 0 eV/atom."
)

In [ ]:
# Let's look at some of these
print(docs[0].formula_pretty)
print(docs[1].formula_pretty)
print(docs[2].formula_pretty)


In [ ]:
# We can restrict to a given "chemical space" via the `chemsys` parameter
with MPRester(api_key=api_key) as mpr:
    docs = mpr.materials.summary.search(chemsys=["Fe-O"], formation_energy=(-1, 0))

In [ ]:
print(docs[0].formula_pretty)
print(docs[-2].formula_pretty)
print(docs[-1].formula_pretty)

In [ ]:
# Sometimes searches are really data-intensive, in which case you should only return what you need
with MPRester(api_key=api_key) as mpr:
    docs = mpr.materials.summary.search(
        elements=["Fe", "O"], fields=["material_id", "structure"]
    )

In [ ]:
# we stored the .structure attribute
docs[0].structure

# Material ID vs. Task ID

Each "material" on the Materials Project is composed of properties from many individual calculations. Each individual calculation is given a unique Task ID, each of which map to a single parent material ID. We can use the `/materials/tasks` end point to find information about these tasks.


In [ ]:
# First, we find the task IDs for each C structure
with MPRester(api_key=api_key) as mpr:
    docs = mpr.materials.summary.search(
        chemsys=["C"], fields=["material_id", "task_ids"]
    )

In [ ]:
for doc in docs:
    print(f"{doc.material_id}: {doc.task_ids}")

In [ ]:
# Let's search all the tasks for the first material we found
with MPRester(api_key=api_key) as mpr:
    task_docs = mpr.materials.tasks.search(task_ids=docs[0].task_ids)

In [ ]:
# The first task doc
task_docs[0]

In [ ]:
# We can even get the input and output parameters for full reproducibility
task_docs[0].input

In [ ]:
task_docs[0].output

In [ ]:
with MPRester(api_key=api_key) as mpr:
    thermo_docs = mpr.materials.thermo.search(formula="PbN6")

In [ ]:
print(thermo_docs[0].formula_pretty)

In [ ]:
thermo_docs[0].decomposes_to

In [ ]:
# N8? What?
with MPRester(api_key=api_key) as mpr:
    n8_doc = mpr.get_structure_by_material_id("mp-25")
n8_doc.to(filename="N8.cif")  # view in VESTA or MatterViz; oh, it's just N2!

In [ ]:
n8_doc.reduced_formula

# ToDo


## Exploration


Now it's your turn. Go to the Materials Project website and pick any material that makes you happy. Take a note of the Material ID. Then, using the MP API's `MPRester.materials.summary.search`, answer the following question:

Using the `.search` method, put together a query that would return your material in the resulting list. The list that is returned is of type `list[SummaryDoc]`. Confirm that you indeed returned the desired structure by iterating through the list and accessing the `.material_id` attribute. Try to make your query as specific as possible so that you are returning only the material(s) of interest.


In [ ]:
with MPRester(api_key=api_key) as mpr:
    materials = mpr.materials.summary.search(
        # input keyword arguments and values here
    )

In [ ]:
# Write a simple code to confirm your desired material has been returned.


## Complex Query


Here is a harder question if you are feeling ambitious. How many materials on the Materials Project are experimentally synthesized (i.e. not theoretical), are **ternary** compounds (i.e. materials with 3 unique elements), and contain **at least Ca and Al**? Bonus: Which one has the lowest density?


In [ ]:
with MPRester(api_key=api_key) as mpr:
    materials = mpr.materials.summary.search(
        # input keyword arguments and values here
    )

In [ ]:
# Write a simple code to determine how many entries are returned

# ToDo Answer Key


## Exploration


The answer here will depend on your material. I chose CsAu. It has Material ID `mp-2667`.


In [ ]:
with MPRester(api_key=api_key) as mpr:
    materials = mpr.materials.summary.search(num_elements=2, elements=["Cs", "Au"])

In [ ]:
for material in materials:
    print(material.material_id)

## Complex Query


We start by making our multi-parameter query. If you aren't sure what keywords are available, you should inspect the definition of the `.search` method or read the MP API documentation.


In [ ]:
with MPRester(api_key=api_key) as mpr:
    materials = mpr.materials.summary.search(
        theoretical=False, num_elements=3, elements=["Ca", "Al"]
    )

Then we can get the length of the returned `list[Structure]` to see how many entries there are:


In [ ]:
materials[0].material_id
print(f"The number of entries is {len(materials)}")

Alright, now how about the lowest density material? We can iterate through the `list[SummaryDoc]` and find the one with the lowest `.density` attribute.


In [ ]:
import numpy as np

density = np.inf
for material in materials:
    if material.density < density:
        lowest_density_material = material
        density = material.density

print(
    f"The lowest density one is {lowest_density_material.material_id} with density {lowest_density_material.density} g/cm^3"
)

If you wanted to do the above code more efficiently:


In [ ]:
idx = np.argmin([material.density for material in materials])
lowest_density_material = materials[idx]
print(
    f"The lowest density one is {lowest_density_material.material_id} with density {lowest_density_material.density} g/cm^3"
)

Finally, we look up the material on the Materials Project to make sure it looks reasonable: https://next-gen.materialsproject.org/materials/mp-1198688?material_ids=mp-1198688


# Being faster and more data-efficient with Materials Project


Knowing how to be data-efficient when using the REST API is good for several reasons. It is helpful for you because it results in much faster results and lower data/memory usage on your side when conducting analyses. It is also much better for the Materials Project because it avoids unnecessary data transfer costs. There are several things you can do to make your queries faster and more data-efficient:

- As already covered, restrict the data returned to the specific fields of interest, to the extent possible:

```python
with MPRester("your_api_key_here") as mpr:
    docs = mpr.materials.summary.search(fields=["material_id", "volume", "elements"])
```

- If you are just exploring / testing queries and don't want to wait for thousands of results to be retrieved, use `num_chunks=1` and `chunk_size=10` parameters when calling `search()` to limit to 10 example results. This works for all searches with all Resters and avoids unnecessary calls:

```python
with MPRester("your_api_key_here") as mpr:
    mpr.summary.search(band_gap=[0,10], num_chunks=1, chunk_size=10)
```

- If you need to get data for many materials, pass the `materials_ids` as a list. This minimizes the number of calls to the API (i.e. don't call `search()` thousands of times!):

```python
with MPRester("your_api_key_here") as mpr:
    docs = mpr.materials.summary.search(material_ids=["mp-149", "mp-13", "mp-22526"])
```

- For more tips, see https://docs.materialsproject.org/downloading-data/using-the-api/tips-for-large-downloads
